In [ ]:
!cp drive/MyDrive/chessBot/NeuralNet.py .
!cp -r drive/MyDrive/chessBot/converted/ .

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tarfile
import os

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)

input_board = tf.keras.Input(shape=(12 * 8 * 8 + 5))

dense_1 = tf.keras.layers.Dense(4096, activation='elu', kernel_initializer='he_normal')(input_board)
dense_2 = tf.keras.layers.Dense(4096, activation='elu', kernel_initializer='he_normal')(dense_1)
dense_3 = tf.keras.layers.Dense(4096, activation='elu', kernel_initializer='he_normal')(dense_2)
dense_4 = tf.keras.layers.Dense(4096, activation='elu', kernel_initializer='he_normal')(dense_3)
dense_5 = tf.keras.layers.Dense(4096, activation='elu', kernel_initializer='he_normal')(dense_4)

output_layer = tf.keras.layers.Dense(64*64, activation='softmax', name='pi')(dense_5)

model = tf.keras.Model(inputs=input_board, outputs=output_layer)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['acc']
)

In [3]:
tmp_model_path = 'drive/MyDrive/chessBot/models/normal_dense/model_dense_elu'
model = tf.keras.models.load_model(tmp_model_path)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 773)]             0         
                                                                 
 dense (Dense)               (None, 4096)              3170304   
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_3 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_4 (Dense)             (None, 4096)              16781312  
                                                                 
 pi (Dense)                  (None, 4096)              167813

In [ ]:
import gc

learning_rate = 0.0000033
tmp_model_path = 'drive/MyDrive/chessBot/models/normal_dense/model_dense_elu_3_epoch'
# tmp_model_path = 'drive/MyDrive/chessBot/models/normal_dense/model_conv_tmp'
tf.keras.backend.set_value(model.optimizer.learning_rate, learning_rate)


def train(data_path):
  gc.collect()
  df = pd.read_csv(data_path, header=None, compression='gzip')
  global learning_rate
  input_positions = df.iloc[:, :-2]
  target_pis = np.asarray(df.iloc[:,-1]).astype('int16')
  model.fit(x = input_positions, y = target_pis, batch_size = 32, epochs = 1, validation_split=0.05)


for i in range(0, 100):
  print(f"Epoch {i}")
  train(f'converted/{i}_converted.csv.gz')
model.save(tmp_model_path)

In [ ]:
import gc

def evaluate():
  for i in range(100, 110):
    data_file_path = f'converted/{i}_converted.csv.gz'
    print(f'Evaluating file {data_file_path}')
    gc.collect()
    df = pd.read_csv(data_file_path, header=None, compression='gzip')
    input_positions = df.iloc[:, :-2]
    target_pis = np.asarray(df.iloc[:,-1]).astype('int16')

    model.evaluate(x = input_positions, y = target_pis, batch_size = 32)

evaluate()

Evaluating file converted/100_converted.csv.gz
15625/15625 [==============================] - 48s 3ms/step - loss: 2.2832 - acc: 0.3816
Evaluating file converted/101_converted.csv.gz
15625/15625 [==============================] - 47s 3ms/step - loss: 2.2911 - acc: 0.3800
Evaluating file converted/102_converted.csv.gz
15625/15625 [==============================] - 47s 3ms/step - loss: 2.2785 - acc: 0.3822
Evaluating file converted/103_converted.csv.gz
15625/15625 [==============================] - 47s 3ms/step - loss: 2.2838 - acc: 0.3809
Evaluating file converted/104_converted.csv.gz
15625/15625 [==============================] - 46s 3ms/step - loss: 2.2905 - acc: 0.3804
Evaluating file converted/105_converted.csv.gz
15625/15625 [==============================] - 46s 3ms/step - loss: 2.2742 - acc: 0.3829
Evaluating file converted/106_converted.csv.gz
15625/15625 [==============================] - 46s 3ms/step - loss: 2.2922 - acc: 0.3805
Evaluating file converted/107_converted.csv.gz
1